<a href="https://colab.research.google.com/github/itssraj/agentic-ai/blob/main/Assingment_solution_XYZ_RAG_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture --no-stderr
%pip install langchain sentence-transformers faiss-cpu openai tiktoken bs4 requests unstructured


In [ ]:
%%capture --no-stderr
%pip install --upgrade --quiet  langchain==0.2.17 langchain-community==0.2.17 langchainhub langchain-chroma
%pip install -qU langchain-openai==0.1.7
!pip gradio_client>=0.16.1
!pip install gradio>=4.44.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 38.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.1/397.1 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 17.9 MB/s eta 0

In [ ]:
# Install required libraries

%%capture --no-stderr
%pip install -qU \
  "requests>=2.32.5" \
  "langgraph>=1.0,<2" \
  "langchain>=1.0.3,<1.1" \
  "langchain-openai>=1.0" \
  "langchain-community>=0.4,<1.0" \
  "langchain-text-splitters>=1.0.0" \
  beautifulsoup4 lxml faiss-cpu langchainhub "gradio>=4.0"

In [ ]:
%pip install -qU "langchain_community>=0.0.308,<1.0"


In [ ]:
%pip install -qU "langchain >=1.0.5"


In [ ]:
# Imports
from langchain_community.document_loaders import UnstructuredURLLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAI
#from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_classic.chains import RetrievalQA
from langchain_classic.chains import create_retrieval_chain
from langchain_openai import ChatOpenAI
import getpass
import os

In [ ]:
# (Optional) Set OpenAI API Key -- Needed if you use OpenAI LLM below
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key (or leave blank if not using): ")

Enter your OpenAI API key (or leave blank if not using): ··········


In [ ]:
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

# Load the Website Content

Fetch the website content and load



In [ ]:
# --- Load Data from URL ---
import bs4
from langchain_community.document_loaders import WebBaseLoader

weather_url = "https://weather.com/weather/tenday/l/New+York+NY+USNY0996:1:US"
loader = WebBaseLoader(weather_url)
bs_kwargs=dict(parse_only=bs4.SoupStrainer(class_=("post-content", "post-title", "post-header")))
documents = loader.load()

# Chunk the Input Document

Use LangChain's  `RecursiveTextSplitter` to split the website content into smaller chunks that fit the model's maximum input length.

In [ ]:
# --- Document Chunking ---
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
chunks = splitter.split_documents(documents)
print(f'Total document chunks: {len(chunks)}')

print(f'Display the chunks: {(chunks)}')

Total document chunks: 25
Display the chunks: [Document(metadata={'source': 'https://weather.com/weather/tenday/l/New+York+NY+USNY0996:1:US', 'title': '10-Day Weather Forecast for Tribeca, Manhattan, New York - The Weather Channel | weather.com', 'description': 'Be prepared with the most accurate 10-day forecast for Tribeca, Manhattan, New York with highs, lows, chance of precipitation from The Weather Channel and Weather.com', 'language': 'en-US'}, page_content='10-Day Weather Forecast for Tribeca, Manhattan, New York - The Weather Channel | weather.com\n\n\n\n\n \n\n\n\n\n\n\n\nHamburger\n\n\n\n\n\nThe Weather Company\n\n\n\n\nHome\n\n\n\n\nHomeHome\nToday\n\n\nTodayToday\nHourly\n\n\n\nHourlyHourly\nDaily\n\n\n10 Day10 Day\nMonthly\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nMonthlyMonthly\nRadar\n\n\nRadarRadar\nVideo\n\n\n\nVideoVideoSevere Weather\n\n\n\nHurricane\n\n\nHurricane CentralHurricane Central\nTornado\n\n\n\n\n\n\n\nTornado CentralTornado Central\nStorm\n\n\n\nSafety & PrepSafety & P

# Generate Embeddings for Each Chunk

Use OpenAIEmbeddings or Hugging Face's Sentence Transformers to generate embeddings for each text chunk.

In [ ]:
# --- Generate Embeddings & Store in FAISS ---
embedder = SentenceTransformerEmbeddings(model_name='all-MiniLM-L6-v2')
vectorstore = FAISS.from_documents(chunks, embedder)

# Retrieve and Answer User Questions

Use LangChain's `PromptTemplate` to query the vector store and retrieve relevant chunks. Prompt the LLM to answer user questions based on the retrieved content.

In [ ]:
#qa = RetrievalQA.from_chain_type(
llm=OpenAI(temperature=0)
#    chain_type='stuff',
retriever=vectorstore.as_retriever(search_kwargs={"k": 4})
#)

In [ ]:
# 2. Incorporate the retriever into a question-answering chain.

from langchain_core.prompts import ChatPromptTemplate
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [ ]:
from langchain_classic.chains.combine_documents import create_stuff_documents_chain

In [ ]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(vectorstore.as_retriever(search_kwargs={"k": 4}), question_answer_chain)

In [ ]:
query = rag_chain.invoke({"input": "What is weather in Newyork?"})
print("query is\n------")
print(query["answer"])
print(query["context"])

query is
------
Currently, in New York, the weather is partly cloudy with a high near 55°F, winds from the west at 5 to 10 mph, and a humidity level of 57%. The UV index is 2 out of 11, and the sunrise is at 6:54 am.


[Document(id='85e0fc06-8640-4936-83b7-bce9c3801c1d', metadata={'source': 'https://weather.com/weather/tenday/l/New+York+NY+USNY0996:1:US', 'title': '10-Day Weather Forecast for Tribeca, Manhattan, New York - The Weather Channel | weather.com', 'description': 'Be prepared with the most accurate 10-day forecast for Tribeca, Manhattan, New York with highs, lows, chance of precipitation from The Weather Channel and Weather.com', 'language': 'en-US'}, page_content='Day55°\nPartly Cloudy Day\n\n\n\n\n\nRain drop\n\n\n13%\n\n\nW 8\xa0mphPartly cloudy skies. High near 55F. Winds W at 5 to 10 mph.\nHumidity\n\n\nHumidity57%\nUV index\n\n\nUV Index2 of 11\nSunrise\n\n\nSunrise6:54 am\nSunset\n\n\nSunset4:31 pmNight41°\nPartly Cloudy Night\n\n\n\n\nRain drop\n\n\n14%\n\n\nNW 6\xa0m

In [ ]:
# --- Example User Query ---
query = 'What is weather in Newyork?'
answer = qa.invoke(query)
print('Answer:', answer)

Answer: {'query': 'What is weather in Newyork?', 'result': ' The weather in New York is currently partly cloudy with a high of 55°F and a low of 41°F. There is a 13% chance of rain and winds are coming from the west at 8 mph. The humidity is at 57% and the UV index is at 2 out of 11. Sunrise is at 6:54 am and sunset is at 4:31 pm.'}


In [ ]:
# Set up the conversational chain with your chosen LLM
from langchain_classic.memory import ConversationBufferMemory
from langchain_classic.chains import ConversationalRetrievalChain
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
conversation = ConversationalRetrievalChain.from_llm(
    llm=OpenAI(temperature=0),                # Or another LLM if preferred
    retriever=vectorstore.as_retriever(),
    memory=memory,
)

In [ ]:
from langchain_classic.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

In [ ]:
from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
        MessagesPlaceholder("chat_history"),
    ]
    )

question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever,
                                   question_answer_chain)

In [ ]:
chat_history = []

query1 = "Is it good to do hiking in Newyork tomorrow"
result1 = conversation({"question": query1, "chat_history": chat_history})
print("System1:", result1["answer"])
chat_history = result1["chat_history"]

print("chat_history", chat_history)

query2 = "When is the best time to do an hiking in Newyork?"
result2 = conversation({"question": query2, "chat_history": chat_history})
print("System2:", result2["answer"])
chat_history = result2["chat_history"]
print("chat_history2", chat_history)

System1:  The best time to go hiking in New York would likely be during the summer or early fall months when the weather is warmer and there is less chance of encountering snow or ice on the trails.
chat_history [HumanMessage(content='Is it good to do hiking in Newyork tomorrow', additional_kwargs={}, response_metadata={}), AIMessage(content=" I don't know, as the context provided does not mention anything about hiking in New York. It is best to check the weather forecast for the specific location and date you are planning to go hiking.", additional_kwargs={}, response_metadata={}), HumanMessage(content='When is the best time to do an hiking in Newyork?', additional_kwargs={}, response_metadata={}), AIMessage(content=' The best time to go hiking in New York would depend on personal preference and the specific location of the hike. However, generally speaking, the spring and fall months (March-May and September-November) tend to have more mild temperatures and less extreme weather condi

In [ ]:
import gradio as gr

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage

def predict(query1, chat_history):
  history_for_llm = []
  for tup in chat_history:
    history_for_llm.extend([HumanMessage(tup[0]), AIMessage(tup[1])])

  ai_msg = rag_chain.invoke({"input": query1, "chat_history": history_for_llm})
  return ai_msg["answer"]

In [ ]:
demo = gr.ChatInterface(fn=predict,
    chatbot=gr.Chatbot(height=300),
    textbox=gr.Textbox(placeholder="Hi I am your virtual assistant, how can I help you today?", container=False, scale=7),
    title="DocumentQABot",
    theme="soft",
    examples=["What is the weather like in NY?", "What is LangSmith?"],)
demo.launch(share=True, debug=True)

/tmp/ipython-input-2714913440.py:2: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot=gr.Chatbot(height=300),
/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:330: UserWarning: The gr.ChatInterface was not provided with a type, so the type of the gr.Chatbot, 'tuples', will be used.
  warnings.warn(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://db0119ce63b8268a43.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://db0119ce63b8268a43.gradio.live
